# Bank Account Manager
Create a class called Account which will be an abstract class for three other classes called CheckingAccount, SavingsAccount and BusinessAccount. Manage credits and debits from these accounts through an ATM style program.

In [8]:
class Account:
    def __init__(self, account_id, owner, balance):
        self.account_id = account_id
        self.owner = owner
        self.balance = balance
        
    def deposit(self, amount):
        if amount <=0:
            raise ValueError('Deposit must be positive')
        else:
            self.balance += amount

    def withdraw(self, amount):
        if amount <=0:
            raise ValueError('Withdrawal must be positive')
        elif amount > self.balance:
            raise ValueError(f'Insufficient funds. Maximum withdrawal: {self.balance}')
        else:
            self.balance -= amount
            print(f'Successful withdrawal. Current balance = {self.balance}')

In [9]:
class CheckingAccount(Account):
    def __init__(self, account_id, owner, balance, overdraft_limit):
        super().__init__(account_id, owner, balance)
        self.overdraft_limit = overdraft_limit
    
    def withdraw(self, amount):
        if amount <=0:
            raise ValueError('Withdrawal must be positive')
        elif self.balance - amount < -self.overdraft_limit:
            raise ValueError("Exceeds overdraft limit.")
        else:
            self.balance -= amount
            print(f'Successful withdrawal. Current balance = {self.balance}')
        

In [10]:
class SavingsAccount(Account):
    # Uses the base class withdraw rule (no overdraft).
    pass

In [11]:
class BusinessAccount(Account):
    def __init__(self, account_id, owner, balance, fee_pct=0.01):  # 1% fee
        super().__init__(account_id, owner, balance)
        self.fee_pct = fee_pct

    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("Withdrawal must be positive.")
        total = amount * (1 + self.fee_pct)   # amount + fee
        # Reuse the base "no overdraft" rule:
        super().withdraw(total)
        print(f"Withdrew {amount} + fee {total-amount:.2f}. Balance = {self.balance:.2f}")
                

In [12]:
class Bank:
    def __init__(self):
        self._accounts = {}
        
    def add_account(self, account: Account):
        if account.account_id in self._accounts:
            raise ValueError("Account id already exists.")
        self._accounts[account.account_id] = account

    def get(self, account_id):
        if account_id not in self._accounts:
            raise KeyError("No such account.")
        return self._accounts[account_id]

    def transfer(self, from_id: str, to_id: str, amount: float) -> None:
        """
        Move money using each account's rules (withdraw may fail).
        """
        src = self.get(from_id)
        dst = self.get(to_id)
        # Withdraw first; if it raises, nothing happens.
        src.withdraw(amount)
        # If withdraw succeeds, deposit to target.
        dst.deposit(amount)

In [13]:
chk = CheckingAccount("C-001", "Ana", balance=100.0, overdraft_limit=200.0)
sav = SavingsAccount("S-001", "Luis", balance=50.0)
biz = BusinessAccount("B-001", "ShopCo", balance=1000.0, fee_pct=0.01)  # 1% fee

chk.deposit(25.0)   # -> 125.0
sav.deposit(50.0)   # -> 100.0
biz.deposit(200.0)  # -> 1200.0

print("Balances after deposits:",
      chk.balance, sav.balance, biz.balance)
# expected: 125.0, 100.0, 1200.0


Balances after deposits: 125.0 100.0 1200.0


In [14]:
chk.withdraw(150.0)           # 125 - 150 = -25 (OK, within -200 limit)
print("Checking after withdraw:", chk.balance)  # expected: -25.0

try:
    chk.withdraw(200.0)       # would go to -225 -> exceeds limit -> error
except ValueError as e:
    print("Checking error:", e)


Successful withdrawal. Current balance = -25.0
Checking after withdraw: -25.0
Checking error: Exceeds overdraft limit.


In [15]:
sav.withdraw(80.0)            # 100 - 80 = 20 (OK)
print("Savings after withdraw:", sav.balance)  # expected: 20.0

try:
    sav.withdraw(25.0)        # 20 - 25 < 0 -> error
except ValueError as e:
    print("Savings error:", e)


Successful withdrawal. Current balance = 20.0
Savings after withdraw: 20.0
Savings error: Insufficient funds. Maximum withdrawal: 20.0


In [16]:
biz.withdraw(100.0)           # total = 100 * 1.01 = 101
print("Business after withdraw:", biz.balance)  # expected: 1200 - 101 = 1099.0

try:
    biz.withdraw(-5)          # negative -> error
except ValueError as e:
    print("Business error:", e)


Successful withdrawal. Current balance = 1099.0
Withdrew 100.0 + fee 1.00. Balance = 1099.00
Business after withdraw: 1099.0
Business error: Withdrawal must be positive.


In [17]:
bank = Bank()
bank.add_account(chk)
bank.add_account(sav)
bank.add_account(biz)

# transfer 40 from checking to savings (uses checking’s overdraft rule)
bank.transfer("C-001", "S-001", 40.0)   # withdraw first, then deposit

print("After transfer (chk -> sav 40):", chk.balance, sav.balance)
# expected: chk: -65.0  (was -25 - 40)
#           sav: 60.0   (was 20 + 40)

# error examples:
try:
    bank.transfer("NOPE", "S-001", 10.0)
except KeyError as e:
    print("Bank error:", e)

try:
    bank.transfer("S-001", "C-001", 1000.0)  # savings can’t overdraft
except ValueError as e:
    print("Bank error:", e)


Successful withdrawal. Current balance = -65.0
After transfer (chk -> sav 40): -65.0 60.0
Bank error: 'No such account.'
Bank error: Insufficient funds. Maximum withdrawal: 60.0
